In [1]:
# !wget https://nyc-tlc.s3.amazonaws.com/trip+data/fhvhv_tripdata_2021-02.csv

--2022-02-23 01:35:42--  https://nyc-tlc.s3.amazonaws.com/trip+data/fhvhv_tripdata_2021-02.csv
Resolving nyc-tlc.s3.amazonaws.com (nyc-tlc.s3.amazonaws.com)... 52.217.133.17
Connecting to nyc-tlc.s3.amazonaws.com (nyc-tlc.s3.amazonaws.com)|52.217.133.17|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 733822658 (700M) [text/csv]
Saving to: ‘fhvhv_tripdata_2021-02.csv’

fhvhv_tripdata_2021 100%[===================>] 699.83M  23.8MB/s    in 30s     

2022-02-23 01:36:13 (23.6 MB/s) - ‘fhvhv_tripdata_2021-02.csv’ saved [733822658/733822658]



In [2]:
import pyspark
from pyspark.sql import SparkSession

In [3]:
spark = SparkSession.builder \
        .master("local[*]") \
        .appName('test') \
        .getOrCreate()

In [4]:
from pyspark.sql import types

In [5]:
!hdfs dfs -copyFromLocal 'fhvhv_tripdata_2021-02.csv'

copyFromLocal: `fhvhv_tripdata_2021-02.csv': File exists


In [6]:
df = spark.read \
    .option("header", "true") \
    .csv('fhvhv_tripdata_2021-02.csv')

In [7]:
df.head(5)

[Row(hvfhs_license_num='HV0003', dispatching_base_num='B02764', pickup_datetime='2021-02-01 00:10:40', dropoff_datetime='2021-02-01 00:21:09', PULocationID='35', DOLocationID='39', SR_Flag=None),
 Row(hvfhs_license_num='HV0003', dispatching_base_num='B02764', pickup_datetime='2021-02-01 00:27:23', dropoff_datetime='2021-02-01 00:44:01', PULocationID='39', DOLocationID='35', SR_Flag=None),
 Row(hvfhs_license_num='HV0005', dispatching_base_num='B02510', pickup_datetime='2021-02-01 00:28:38', dropoff_datetime='2021-02-01 00:38:27', PULocationID='39', DOLocationID='91', SR_Flag=None),
 Row(hvfhs_license_num='HV0005', dispatching_base_num='B02510', pickup_datetime='2021-02-01 00:43:37', dropoff_datetime='2021-02-01 01:23:20', PULocationID='91', DOLocationID='228', SR_Flag=None),
 Row(hvfhs_license_num='HV0003', dispatching_base_num='B02872', pickup_datetime='2021-02-01 00:08:42', dropoff_datetime='2021-02-01 00:17:57', PULocationID='126', DOLocationID='250', SR_Flag=None)]

In [8]:
df.printSchema()

root
 |-- hvfhs_license_num: string (nullable = true)
 |-- dispatching_base_num: string (nullable = true)
 |-- pickup_datetime: string (nullable = true)
 |-- dropoff_datetime: string (nullable = true)
 |-- PULocationID: string (nullable = true)
 |-- DOLocationID: string (nullable = true)
 |-- SR_Flag: string (nullable = true)



In [51]:
schema = types.StructType([
    types.StructField('hvfhs_license_num', types.StringType(), True),
    types.StructField('dispatching_base_num', types.StringType(), True),
    types.StructField('pickup_datetime', types.TimestampType(), True),
    types.StructField('dropoff_datetime', types.TimestampType(), True),
    types.StructField('PULocationID', types.IntegerType(), True),
    types.StructField('DOLocationID', types.IntegerType(), True),
    types.StructField('SR_Flag', types.StringType(), True)
])

In [52]:
df = spark.read \
    .option("header", "true") \
    .schema(schema) \
    .csv('fhvhv_tripdata_2021-02.csv')

In [53]:
df.printSchema()

root
 |-- hvfhs_license_num: string (nullable = true)
 |-- dispatching_base_num: string (nullable = true)
 |-- pickup_datetime: timestamp (nullable = true)
 |-- dropoff_datetime: timestamp (nullable = true)
 |-- PULocationID: integer (nullable = true)
 |-- DOLocationID: integer (nullable = true)
 |-- SR_Flag: string (nullable = true)



In [79]:
spark.version

'3.1.2'

'3.1.2'

In [54]:
df = df.repartition(24)

In [55]:
!ls -lh fhvhv/2021/02

total 209M
-rw-r--r-- 1 root root    0 Feb 23 03:12 _SUCCESS
-rw-r--r-- 1 root root 8.7M Feb 23 03:12 part-00000-64af185b-ea8b-4e14-b061-2917ba39b75b-c000.snappy.parquet
-rw-r--r-- 1 root root 8.7M Feb 23 03:12 part-00001-64af185b-ea8b-4e14-b061-2917ba39b75b-c000.snappy.parquet
-rw-r--r-- 1 root root 8.7M Feb 23 03:12 part-00002-64af185b-ea8b-4e14-b061-2917ba39b75b-c000.snappy.parquet
-rw-r--r-- 1 root root 8.7M Feb 23 03:12 part-00003-64af185b-ea8b-4e14-b061-2917ba39b75b-c000.snappy.parquet
-rw-r--r-- 1 root root 8.7M Feb 23 03:12 part-00004-64af185b-ea8b-4e14-b061-2917ba39b75b-c000.snappy.parquet
-rw-r--r-- 1 root root 8.7M Feb 23 03:12 part-00005-64af185b-ea8b-4e14-b061-2917ba39b75b-c000.snappy.parquet
-rw-r--r-- 1 root root 8.7M Feb 23 03:12 part-00006-64af185b-ea8b-4e14-b061-2917ba39b75b-c000.snappy.parquet
-rw-r--r-- 1 root root 8.7M Feb 23 03:12 part-00007-64af185b-ea8b-4e14-b061-2917ba39b75b-c000.snappy.parquet
-rw-r--r-- 1 root root 8.7M Feb 23 03:12 part-00008-64af185b-ea8b-4

total 209M

In [69]:
df.write.parquet('fhvhv/2021/02/', mode='overwrite')

In [71]:
!hdfs dfs -copyToLocal 'fhvhv'

copyToLocal: `fhvhv/2021/02/_SUCCESS': File exists


In [72]:
df_fhvhv = spark.read.parquet('fhvhv/2021/02/*')

In [73]:
df_fhvhv.printSchema()

root
 |-- hvfhs_license_num: string (nullable = true)
 |-- dispatching_base_num: string (nullable = true)
 |-- pickup_datetime: timestamp (nullable = true)
 |-- dropoff_datetime: timestamp (nullable = true)
 |-- PULocationID: integer (nullable = true)
 |-- DOLocationID: integer (nullable = true)
 |-- SR_Flag: string (nullable = true)



### Question 3. Count records

How many taxi trips were there on February 15?

Consider only trips that started on February 15.

In [74]:
df_fhvhv.registerTempTable('fhvhv_data')

In [75]:
spark.sql("""
SELECT
    date_trunc('day', pickup_datetime) AS date,
    count(1) AS trips
FROM
    fhvhv_data
WHERE
    pickup_datetime = '2021-02-15'
GROUP BY 
    date
""").show()

+-------------------+-----+
|               date|trips|
+-------------------+-----+
|2021-02-15 00:00:00|    5|
+-------------------+-----+




|               date|trips|
|-------------------|-----|
|2021-02-15 00:00:00|    5|

### Question 4. Longest trip for each day

Now calculate the duration for each trip.

Trip starting on which day was the longest?

In [96]:
spark.sql("""
SELECT
    date_trunc('second', pickup_datetime) AS pickup_datetime,
    date_trunc('second', dropoff_datetime),
    INT(TO_UNIX_TIMESTAMP(dropoff_datetime, 'yyyy-mm-dd s-m-h') - TO_UNIX_TIMESTAMP(pickup_datetime, 'yyyy-mm-dd s-m-h')) AS interval,
    dropoff_datetime - pickup_datetime AS t_interval
FROM
    fhvhv_data
GROUP BY
    pickup_datetime, dropoff_datetime
ORDER BY
    3 DESC, 1
LIMIT
    3
""").show()

+-------------------+------------------------------------+--------+--------------------+
|    pickup_datetime|date_trunc(second, dropoff_datetime)|interval|          t_interval|
+-------------------+------------------------------------+--------+--------------------+
|2021-02-11 13:40:44|                 2021-02-12 10:39:44|   75540| 20 hours 59 minutes|
|2021-02-17 15:54:53|                 2021-02-18 07:48:34|   57221|15 hours 53 minut...|
|2021-02-20 12:08:15|                 2021-02-21 00:22:14|   44039|12 hours 13 minut...|
+-------------------+------------------------------------+--------+--------------------+



|    pickup_datetime|date_trunc(second, dropoff_datetime)|interval|          t_interval|
|-------------------|------------------------------------|--------|--------------------|
|2021-02-11 13:40:44|                 2021-02-12 10:39:44|   75540| 20 hours 59 minutes|
|2021-02-17 15:54:53|                 2021-02-18 07:48:34|   57221|15 hours 53 minut...|
|2021-02-20 12:08:15|                 2021-02-21 00:22:14|   44039|12 hours 13 minut...|

### Question 5. Most frequent dispatching_base_num

Now find the most frequently occurring dispatching_base_num in this dataset.

How many stages this spark job has?

Note: the answer may depend on how you write the query, so there are multiple correct answers. Select the one you have.

In [99]:
spark.sql("""
SELECT
    dispatching_base_num,
    COUNT(1) AS ocurrencies
FROM
    fhvhv_data
GROUP BY
    dispatching_base_num
ORDER BY
    ocurrencies DESC
LIMIT
    3
""").show()

+--------------------+-----------+
|dispatching_base_num|ocurrencies|
+--------------------+-----------+
|              B02510|    3233664|
|              B02764|     965568|
|              B02872|     882689|
+--------------------+-----------+



[Stage 103:======================================>                  (2 + 1) / 3]

|dispatching_base_num|ocurrencies|
|--------------------|-----------|
|              B02510|    3233664|
|              B02764|     965568|
|              B02872|     882689|
|--------------------|-----------|

### Question 6. Most common locations pair

Find the most common pickup-dropoff pair.

For example:

"Jamaica Bay / Clinton East"

Enter two zone names separated by a slash

If any of the zone names are unknown (missing), use "Unknown". For example, "Unknown / Clinton East"

In [101]:
!wget https://s3.amazonaws.com/nyc-tlc/misc/taxi+_zone_lookup.csv -O taxi_zone_lookup.csv

--2022-02-25 01:29:13--  https://s3.amazonaws.com/nyc-tlc/misc/taxi+_zone_lookup.csv
Resolving s3.amazonaws.com (s3.amazonaws.com)... 52.216.165.77
Connecting to s3.amazonaws.com (s3.amazonaws.com)|52.216.165.77|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 12322 (12K) [application/octet-stream]
Saving to: ‘taxi_zone_lookup.csv’

taxi_zone_lookup.cs 100%[===================>]  12.03K  --.-KB/s    in 0s      

2022-02-25 01:29:13 (88.3 MB/s) - ‘taxi_zone_lookup.csv’ saved [12322/12322]



In [103]:
!hdfs dfs -copyFromLocal 'taxi_zone_lookup.csv'

In [104]:
df_zone_lookup = spark.read \
    .option("header", "true") \
    .csv('taxi_zone_lookup.csv')

In [108]:
df_zone_lookup.printSchema()

root
 |-- LocationID: string (nullable = true)
 |-- Borough: string (nullable = true)
 |-- Zone: string (nullable = true)
 |-- service_zone: string (nullable = true)



In [148]:
df_fhvhv.printSchema()

root
 |-- hvfhs_license_num: string (nullable = true)
 |-- dispatching_base_num: string (nullable = true)
 |-- pickup_datetime: timestamp (nullable = true)
 |-- dropoff_datetime: timestamp (nullable = true)
 |-- PULocationID: integer (nullable = true)
 |-- DOLocationID: integer (nullable = true)
 |-- SR_Flag: string (nullable = true)



In [154]:
df_zone_lookup.registerTempTable('zone_lookup')

In [152]:
df_fhvhv.registerTempTable('fhvhv_data')

In [192]:
spark.sql("""
SELECT
    pickup_zone.Zone AS pickup_zone_name,
    dropoff_zone.Zone AS dropzone_name,
    CONCAT(pickup_zone.Zone, '/', dropoff_zone.Zone) AS location_pair,
    COUNT(fhvhv_data.pickup_datetime) AS trips

FROM
    fhvhv_data 

INNER JOIN 
    zone_lookup AS pickup_zone ON 
    fhvhv_data.PULocationID = pickup_zone.LocationID

INNER JOIN 
    zone_lookup AS dropoff_zone ON 
    fhvhv_data.DOLocationID = dropoff_zone.LocationID

GROUP BY
    2, 1

ORDER BY
    4 DESC


LIMIT
    5
""").show(20, False)

+-------------------+-------------------+---------------------------------------+-----+
|pickup_zone_name   |dropzone_name      |location_pair                          |trips|
+-------------------+-------------------+---------------------------------------+-----+
|East New York      |East New York      |East New York/East New York            |45041|
|Borough Park       |Borough Park       |Borough Park/Borough Park              |37329|
|Canarsie           |Canarsie           |Canarsie/Canarsie                      |28026|
|Crown Heights North|Crown Heights North|Crown Heights North/Crown Heights North|25976|
|Bay Ridge          |Bay Ridge          |Bay Ridge/Bay Ridge                    |17934|
+-------------------+-------------------+---------------------------------------+-----+



|pickup_zone_name   |dropzone_name      |location_pair                          |trips|
|-------------------|-------------------|---------------------------------------|-----|
|East New York      |East New York      |East New York/East New York            |45041|
|Borough Park       |Borough Park       |Borough Park/Borough Park              |37329|
|Canarsie           |Canarsie           |Canarsie/Canarsie                      |28026|
|Crown Heights North|Crown Heights North|Crown Heights North/Crown Heights North|25976|
|Bay Ridge          |Bay Ridge          |Bay Ridge/Bay Ridge                    |17934|